In [7]:
import pretty_midi

In [8]:
import music21

In [9]:
import os

In [10]:
import pandas as pd

In [19]:
import numpy as np

In [20]:
def piano_roll_to_pretty_midi(piano_roll, fs=100, program=0):
    '''Convert a Piano Roll array into a PrettyMidi object
     with a single instrument.
    Parameters
    ----------
    piano_roll : np.ndarray, shape=(128,frames), dtype=int
        Piano roll of one instrument
    fs : int
        Sampling frequency of the columns, i.e. each column is spaced apart
        by ``1./fs`` seconds.
    program : int
        The program number of the instrument.
    Returns
    -------
    midi_object : pretty_midi.PrettyMIDI
        A pretty_midi.PrettyMIDI class instance describing
        the piano roll.
    '''
    notes, frames = piano_roll.shape
    pm = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=program)

    # pad 1 column of zeros so we can acknowledge inital and ending events
    piano_roll = np.pad(piano_roll, [(0, 0), (1, 1)], 'constant')

    # use changes in velocities to find note on / note off events
    velocity_changes = np.nonzero(np.diff(piano_roll).T)

    # keep track on velocities and note on times
    prev_velocities = np.zeros(notes, dtype=int)
    note_on_time = np.zeros(notes)

    for time, note in zip(*velocity_changes):
        # use time + 1 because of padding above
        velocity = piano_roll[note, time + 1]
        time = time / fs
        if velocity > 0:
            if prev_velocities[note] == 0:
                note_on_time[note] = time
                prev_velocities[note] = velocity
        else:
            pm_note = pretty_midi.Note(
                    velocity=prev_velocities[note],
                    pitch=note,
                    start=note_on_time[note],
                    end=time)
            instrument.notes.append(pm_note)
            prev_velocities[note] = 0
    pm.instruments.append(instrument)
    return pm

In [36]:
midi = pretty_midi.PrettyMIDI("D:/data/thesis_model2/MIDI_tests/1.mid")

In [37]:
midi.get_tempo_changes()

(array([0.]), array([120.]))

`pianoroll = midi.get_piano_roll(fs=48)` to achieve the same sampling as pypianoroll library

In [68]:
pianoroll = midi.get_piano_roll(fs=48)

df = pd.DataFrame(pianoroll)

df.loc[70:80, :30]

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
70,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
71,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0
72,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,80.0,...,80.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
74,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
75,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
76,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
77,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
78,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
reconstructed_pianoroll = piano_roll_to_pretty_midi(df.loc[:,70:], 10)

In [102]:
reconstructed_pianoroll.write("reconstructed_pianoroll.mid")

In [114]:
track = midi.instruments[0]

In [115]:
track.notes

[Note(start=0.000000, end=0.473958, pitch=72, velocity=80),
 Note(start=0.500000, end=0.973958, pitch=72, velocity=80),
 Note(start=1.000000, end=1.473958, pitch=72, velocity=80),
 Note(start=1.500000, end=1.973958, pitch=72, velocity=80),
 Note(start=2.000000, end=2.948958, pitch=74, velocity=80),
 Note(start=3.000000, end=3.948958, pitch=74, velocity=80),
 Note(start=4.000000, end=4.948958, pitch=74, velocity=80),
 Note(start=5.000000, end=5.948958, pitch=72, velocity=80),
 Note(start=7.000000, end=7.948958, pitch=72, velocity=80),
 Note(start=8.000000, end=8.948958, pitch=72, velocity=80)]

In [146]:
midi = pretty_midi.PrettyMIDI("D:\\data\\thesis_model2\\MIDI_tests\\2.mid")

In [147]:
track = midi.instruments[0]

In [148]:
track.notes

[Note(start=0.000000, end=0.473958, pitch=72, velocity=80),
 Note(start=0.500000, end=0.973958, pitch=72, velocity=80),
 Note(start=1.000000, end=1.473958, pitch=72, velocity=80),
 Note(start=0.000000, end=1.973958, pitch=69, velocity=80),
 Note(start=1.500000, end=1.973958, pitch=72, velocity=80),
 Note(start=2.000000, end=2.947917, pitch=74, velocity=80),
 Note(start=3.000000, end=3.947917, pitch=74, velocity=80),
 Note(start=4.000000, end=4.947917, pitch=74, velocity=80),
 Note(start=5.000000, end=5.947917, pitch=72, velocity=80),
 Note(start=7.000000, end=7.947917, pitch=72, velocity=80),
 Note(start=8.000000, end=8.947917, pitch=72, velocity=80)]

# Oore encoding

In [116]:
events = []

## Timesteps

In [140]:
step_for_timesteps = (1.0008-0.008)/125

timesteps = np.arange(0.008, 1.0001, step_for_timesteps)

timesteps = np.around(timesteps, decimals=3)
assert len(timesteps) == 125

In [ ]:
for i, note in enumerate(tracks.notes):
    

# MusicXML

In [58]:
c = music21.converter.parse('D://data//chord-melody-dataset//aint_no_mountain/f.xml')
# c.show('musicxml.pdf')
# c.show('midi')

In [59]:
pars_stream = c.parts.stream()

In [60]:
pars_stream.classes

('Score', 'Stream', 'StreamCoreMixin', 'Music21Object', 'object')

In [61]:
for p in pars_stream:
    print(p.id)

Nylon Guitar
